In [1]:
%%capture
%pip install llama-index llama-index-embeddings-openai qdrant-client llama-index-vector-stores-qdrant llama-index llama-index-llms-openai llama-index-vector-stores-faiss faiss-cpu llama-index-llms-anthropic tavily-python llama-index-experimental

In [11]:
import os
import nest_asyncio
from getpass import getpass
from dotenv import load_dotenv

# Import LlamaIndex components
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.workflow import Event, Workflow, Context, StopEvent, step
from llama_index.core.workflow import StartEvent


# Load environment variables and apply nest_asyncio for async operations
load_dotenv()
nest_asyncio.apply()

# Get API keys
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY') or getpass("Enter OPENAI_API_KEY: ")

# Initialize LLM
llm = OpenAI(model="gpt-4o-2024-11-20", api_key=OPENAI_API_KEY, temperature=0.5, max_tokens=512)

In [ ]:
import pandas as pd
import re
from llama_index.experimental.query_engine import PandasQueryEngine 


class DataPrepEvent(Event):
    original_path: str
    column_names: list[str]
    
class DataAnalysisEvent(Event):
    prepared_data_description: str
    original_path: str

# Define our multi-agent workflow
class DataAnalysisFlow(Workflow):
    
    @step
    async def setup(self, ctx: Context, ev: StartEvent) -> DataPrepEvent:
        """Initialize the agents and setup the workflow"""
        # Store the agents from the StartEvent
        self.data_prep_agent = ev.data_prep_agent
        self.data_analysis_agent = ev.data_analysis_agent
        
       
        # --- Load data and create Pandas Query Engine ---
        try:
            df = pd.read_csv(ev.dataset_path)
            # Handle potential issues like missing values before creating the engine if needed
            # df = df.fillna('NA') # Example: fill NaNs, adjust as necessary
            
            # Create the query engine, passing the LLM is recommended
            query_engine = PandasQueryEngine(df=df, llm=llm, verbose=True) 
            
            # Store the DataFrame and query engine in the context
            await ctx.set("dataframe", df)
            await ctx.set("query_engine", query_engine)
            await ctx.set("original_path", ev.dataset_path)
            
            print(f"Successfully loaded {ev.dataset_path} and created PandasQueryEngine.")
            
            # Return event with basic info for the next step
            return DataPrepEvent(
                original_path=ev.dataset_path, 
                column_names=df.columns.tolist()
            )
        except Exception as e:
            print(f"Error during setup: Failed to load {ev.dataset_path} or create engine. Error: {e}")
            # Need to decide how to handle errors - perhaps raise or return a specific error event
            # For now, returning an empty event or raising might be options. Let's raise.
            raise ValueError(f"Setup failed: {e}")
        
    @step
    async def data_preparation(self, ctx: Context, ev: DataPrepEvent) -> DataAnalysisEvent:
        """Use the data prep agent to suggest cleaning/preparation based on schema."""
        query_engine: PandasQueryEngine = await ctx.get("query_engine")
        df: pd.DataFrame = await ctx.get("dataframe")
        
        # Get more comprehensive initial info using the engine
        try:
            # --- CHANGE: Use describe(include='all') ---
            initial_info = str(query_engine.query("Show the shape of the dataframe (number of rows and columns) and the output of df.describe(include='all')"))
            print(f"--- Initial Info for Prep Agent ---\n{initial_info}\n------------------------------------") 
        except Exception as e:
            print(f"Warning: Could not query initial info from engine: {e}")
            initial_info = f"Columns: {ev.column_names}"

        # Ask the data preparation agent for preparation steps/description
        # --- CHANGE: Refined Prompt ---
        prep_prompt = (
            f"The dataset (from {ev.original_path}) has the following shape and summary statistics:\n{initial_info}\n\n" 
            f"Based *only* on these statistics, describe the necessary data preparation steps. "
            f"Specifically mention potential issues like outliers (e.g., in 'Distance' max value), missing values (e.g., count mismatch in 'Time'), "
            f"and data quality issues in categorical columns (e.g., unique count vs expected for 'Mode', potential typos like 'Bas', 'Cra', 'Walt'). "
            f"Suggest specific actions like imputation for 'Time', outlier investigation for 'Distance', and checking unique values/correcting typos in 'Mode'. "
            f"Focus on describing *what* needs to be done and *why* based *strictly* on the provided stats. If no issues are apparent from the stats, state that clearly. ALWAYS provide a description."
        )
        result = self.data_prep_agent.chat(prep_prompt)
        
        prepared_data_description = None # Initialize
        if hasattr(result, 'response'):
            prepared_data_description = result.response
            # --- CHANGE: Add check for None/empty response ---
            if not prepared_data_description: 
                prepared_data_description = "Agent returned an empty description despite the prompt."
                print("Warning: Agent response attribute was empty.")
            # --- END CHANGE ---
        else:
            # Fallback in case the response structure is different
            prepared_data_description = "Could not extract data preparation description from agent response."
            print(f"Warning: Agent response does not have expected 'response' attribute. Full result: {result}")
        
        # Optional: Print the description generated by the agent
        print(f"--- Prep Agent Description Output ---\n{prepared_data_description}\n------------------------------------")
        
        await ctx.set("prepared_data_description", prepared_data_description)
        
        return DataAnalysisEvent(
            prepared_data_description=prepared_data_description,
            original_path=ev.original_path
        )
    
    @step
    async def data_analysis(self, ctx: Context, ev: DataAnalysisEvent) -> StopEvent:
        """Use the analysis agent to generate and execute queries via the PandasQueryEngine."""
        query_engine: PandasQueryEngine = await ctx.get("query_engine")
        
        # Updated analysis query request prompt
        analysis_query_request = (
            f"Based on the dataset (from {ev.original_path}) and the following preparation description:\n"
            f"<preparation_description>\n{ev.prepared_data_description}\n</preparation_description>\n\n"
            f"Generate a concise analysis plan (what key insights to look for). "
            # --- CHANGE: Added instruction for complete queries ---
            f"Then, provide 2-3 specific, COMPLETE, single-line pandas queries (using the 'df' variable) "
            f"to execute via a query engine to find these insights. Ensure each query is valid pandas code. "
            f"Enclose each query strictly within <query> and </query> tags. "
            f"Example: <query>df.describe(include='all')</query>\n<query>df.groupby('Mode')['Time'].mean()</query>"
        )
        
        agent_response = self.data_analysis_agent.chat(analysis_query_request)

        # --- CHANGE: Properly extract response text ---
        agent_response_text = "Agent did not provide a valid analysis plan." # Default fallback
        if hasattr(agent_response, 'response') and agent_response.response:
             agent_response_text = agent_response.response
        else:
             print(f"Warning: Agent response does not have expected 'response' attribute. Full result: {agent_response}")
        # --- END CHANGE ---

        print(f"--- Agent suggested analysis plan and queries ---\n{agent_response_text}\n-------------------------------------------------") 
        
        # Use regex to find queries within <query> tags
        potential_queries = re.findall(r"<query>(.*?)</query>", agent_response_text) # Use checked text

        # --- CHANGE: Extract plan separately for cleaner final output ---
        # Attempt to extract text before the first query tag as the plan
        plan_text = agent_response_text.split('<query>')[0].strip()
        if not plan_text: # Fallback if split fails or plan is empty
            plan_text = "Analysis plan not explicitly extracted."
        # --- END CHANGE ---

        analysis_results_summary = f"Agent Analysis Plan:\n{plan_text}\n\nQuery Engine Results:\n" # Use extracted plan

        if not potential_queries:
             # Fallback if no tagged queries are found
             print("Warning: No queries found within <query> tags. Falling back to df.describe().")
             potential_queries.append("df.describe()") 

        print(f"--- Attempting to execute queries: {potential_queries} ---")
        executed_queries_count = 0
        # ... (rest of the query execution loop remains the same) ...

        await ctx.set("analysis_results", analysis_results_summary)
        
        # Return a StopEvent with the final result description
        return StopEvent(result={
            "prepared_data_description": ev.prepared_data_description,
            "analysis_results": analysis_results_summary # Contains plan and query results
        })
    

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.core.workflow import Context


async def execute_pandas_query(ctx: Context, query_str: str) -> str:
    """
    Executes a single-line pandas query string against the DataFrame.
    Use this tool to perform data analysis tasks like describing data,
    grouping, filtering, calculating means, finding unique values, etc.

    Args:
        query_str (str): The pandas query to execute (e.g., "df.describe()", "df['Mode'].unique()").
                         Must use 'df' as the DataFrame variable.
    """
    try:
        # Retrieve the query engine from the context
        query_engine: PandasQueryEngine = await ctx.get("query_engine")
        if not query_engine:
            return "Error: PandasQueryEngine not found in context."

        print(f"Tool executing query: {query_str}")
        response = query_engine.query(query_str)
        result = str(response)
        print(f"Tool query result: {result[:500]}...") # Print truncated result
        return result
    except Exception as e:
        print(f"Tool query error: {e}")
        return f"Error executing query '{query_str}': {e}"


pandas_query_tool = FunctionTool.from_defaults(fn=execute_pandas_query)

In [ ]:
# Create the agents - 
def create_agents():
    """Create and return the agents needed for our workflow"""
    data_prep_agent = FunctionCallingAgent.from_tools(
        tools=[],
        llm=llm,
        verbose=False,
        system_prompt="You are a data preparation agent. Your job is to describe the necessary steps to clean, transform, and prepare data for analysis based on provided statistics. "
                     "You handle tasks like dealing with missing values, normalizing data, feature engineering, and ensuring data quality."
    )
    
    data_analysis_agent = FunctionCallingAgent.from_tools(
        tools=[pandas_query_tool],
        llm=llm,
        verbose=True,
        # Updated system prompt
        system_prompt=(
            "You are a data analysis agent. Your job is to analyze data based on a preparation description. "
            "You identify patterns, correlations, and insights. "
            "Use the 'execute_pandas_query' tool to run pandas queries against the DataFrame ('df') to find answers. "
            "Plan your analysis steps and then use the tool to execute necessary queries. "
            "Summarize your findings based on the tool's results."
        )
    )
    
    return data_prep_agent, data_analysis_agent

In [16]:
async def run_workflow(dataset_path):
    """Run the data analysis workflow on the given dataset"""
    # Create the agents
    data_prep_agent, data_analysis_agent = create_agents()
    
    # Initialize the workflow
    # Increased timeout as query engine + LLM calls can take longer
    workflow = DataAnalysisFlow(timeout=180, verbose=True) 
    
    # Run the workflow - still pass dataset_path to StartEvent
    try:
        handler = workflow.run(
            dataset_path=dataset_path,
            data_prep_agent=data_prep_agent,
            data_analysis_agent=data_analysis_agent
            # No need to pass content here, setup handles loading
        )
        
        # Get the final result
        final_result = await handler
        
        print("\n==== Analysis Results ====")
        # Adjust keys based on the StopEvent result dictionary
        prep_desc = final_result.get('prepared_data_description', 'N/A')
        analysis_res = final_result.get('analysis_results', 'N/A')
        
        print(f"1. Data Preparation Description:\n{prep_desc}...\n") # Show more potentially
        print(f"2. Analysis Results (Plan & Queries):\n{analysis_res}...\n") # Show more potentially
        
        return final_result
    except Exception as e:
         print(f"Workflow failed: {e}")
         return None


In [17]:
# Example: Run the workflow with a CSV file
# You'll need to update this with your actual dataset path
dataset_path = r"C:\Users\anteb\Desktop\Courses\Projects\data_analysis_ai\data_analysis_agent\Commute_Times_V1.csv"

# Execute the workflow
await run_workflow(dataset_path)

Running step setup
Successfully loaded C:\Users\anteb\Desktop\Courses\Projects\data_analysis_ai\data_analysis_agent\Commute_Times_V1.csv and created PandasQueryEngine.
Step setup produced event DataPrepEvent
Running step data_preparation
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
(df.shape, df.describe(include='all'))
```
> Pandas Output: ((281, 4),               Case Mode    Distance        Time
count   281.000000  281  281.000000  278.000000
unique         NaN    9         NaN         NaN
top            NaN  Car         NaN         NaN
freq           NaN   84         NaN         NaN
mean    140.978648  NaN    3.658007   19.622302
std      81.287714  NaN    8.206031   13.720435
min       1.000000  NaN    0.200000    2.000000
25%      71.000000  NaN    1.700000   10.000000
50%     141.000000  NaN    3.000000   16.000000
75%     211.0

{'prepared_data_description': "Based on the provided summary statistics of the dataset, here are the necessary data preparation steps:\n\n### 1. **Handling Missing Values in 'Time' Column**\n   - **Issue**: The 'Time' column has 278 non-missing values, while the dataset has 281 rows. This indicates 3 missing values.\n   - **Action**: Impute the missing values in the 'Time' column. Since the 'Time' column appears to be continuous and skewed (mean is 19.62 and median is 16), use the median (16.0) for imputation to avoid being influenced by potential outliers.\n   - **Reason**: Imputation ensures no rows are excluded during analysis due to missing values.\n\n---\n\n### 2. **Investigating Outliers in 'Distance'**\n   - **Issue**: The maximum value for 'Distance' is 99, which is significantly higher than the 75th percentile (4.2). This suggests the presence of a potential outlier or data entry error.\n   - **Action**: Investigate the rows with 'Distance' values near the maximum (e.g., >75th